In [196]:
import os
from dotenv import load_dotenv

In [197]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [198]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [199]:
from langchain.document_loaders import PyPDFLoader

In [200]:
%cd ..

f:\


f:\Langchain\Interview_Ques_Creator\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
file_path = "data/DS_Book.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [202]:
len(data)

38

In [203]:
question_gen = ""
for page in data:
    question_gen += page.page_content

In [204]:
question_gen

'Basics of Data ScienceBasics of Data Science\nDr.K.Pazhanikumar\nHead\nDepartment of Computer Science\nS.T.Hindu CollegeWhat is Data Science?What is Data Science?\nData science is the study of data to extract meaningful\ninsights for business. It is a multidisciplinary approach that\ncombines principles and practices from the fields of\nmathematics, statistics, artificial intelligence, and computer\nengineering to analyze large amounts of data.\nIn Wikipedia, Data Science is defined as a scientific field\nthat uses scientific methods to extract knowledge and insights from\nstructured and unstructured data, and apply knowledge and\nactionable insights from data across a broad range of application\ndomains.Why Data Science?Why Data Science?\n\uf097 Data is the oil for today’s world. With the right tools,\ntechnologies, algorithms, we can use data and convert it\ninto a distinct business advantage\n\uf097 Data Science can help us to detect fraud using advanced\nmachine learning algorithm

In [205]:
from langchain.text_splitter import TokenTextSplitter


In [206]:
spliiter_ques_gen = TokenTextSplitter(
    model_name = "gpt-3.5-turbo",
    chunk_size = 1000,
    chunk_overlap = 200
)

In [207]:
chunk_ques_gen = spliiter_ques_gen.split_text(question_gen)
len(chunk_ques_gen)

3

### Converting the list to a document then try to convert into chunks

In [208]:
from langchain.docstore.document import Document

In [209]:
document_ques_gen = [Document(page_content= t) for t in chunk_ques_gen]

In [210]:
spliiter_ans_gen = TokenTextSplitter(
    model_name = "gpt-3.5-turbo",
    chunk_size = 1000,
    chunk_overlap = 200
)

In [211]:
document_answer_gen = spliiter_ans_gen.split_documents(
    document_ques_gen
)

In [212]:
len(document_answer_gen)

3

In [213]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [214]:
llm_ques_gen_pipeline = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.7, 
    top_p=0.85)

In [215]:
prompt_template = """
You are an expert at creating questions based on Data Science material and documentation.
Your goal is to prepare a Data Science enthusiasts for their Data Science exam and  coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the Data Science enthusiasts for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [216]:
from langchain.prompts import PromptTemplate

In [217]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [218]:
refine_template = ("""
You are not an expert at creating practice questions based on Data Science material and documentation.
Your goal is to help a Data Science enthusiast prepare for a Data Science test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------
                   
Given the new context, refine the original questions in English. 
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [219]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template
)

In [220]:
from langchain.chains.summarize import load_summarize_chain

In [221]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline,
                                      chain_type = "refine",
                                      verbose = True,
                                      question_prompt = PROMPT_QUESTIONS,
                                      refine_prompt = REFINE_PROMPT_QUESTIONS
                                      )

In [222]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on Data Science material and documentation.
Your goal is to prepare a Data Science enthusiasts for their Data Science exam and  coding tests.
You do this by asking questions about the text below:

------------
Basics of Data ScienceBasics of Data Science
Dr.K.Pazhanikumar
Head
Department of Computer Science
S.T.Hindu CollegeWhat is Data Science?What is Data Science?
Data science is the study of data to extract meaningful
insights for business. It is a multidisciplinary approach that
combines principles and practices from the fields of
mathematics, statistics, artificial intelligence, and computer
engineering to analyze large amounts of data.
In Wikipedia, Data Science is defined as a scientific field
that uses scientific methods to extract knowledge and insights from
structured and unstructured data, and apply knowledge and
actionable

In [223]:
# from langchain.embeddings import OllamaEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [224]:
# embeddings = OllamaEmbeddings(model="models/llama-3")
embeddings=GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [225]:
from langchain.vectorstores import FAISS

In [226]:
vector_store = FAISS.from_documents(document_answer_gen, embeddings)


In [227]:
llm_ans_gen = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7)

In [ ]:
ques_list = ques.split("\n")
ques_list

['**Refined Questions:**',
 '',
 '**Question 1:**',
 '',
 'Define Data Science and elaborate on its multidisciplinary nature, highlighting the key fields that contribute to its foundation.',
 '',
 '**Question 2:**',
 '',
 'Explain the significance of Data Science in business decision-making. Provide specific examples of how it has been applied to solve real-world business problems.',
 '',
 '**Question 3:**',
 '',
 'List and describe the various disciplines related to Data Science, including their primary focus and how they complement each other.',
 '',
 '**Question 4:**',
 '',
 'Distinguish between structured, unstructured, and semi-structured data types, providing examples of each and discussing their implications for Data Science analysis.',
 '',
 '**Question 5:**',
 '',
 'Define digitalization and digital transformation. Explain how they relate to Data Science and how they have impacted the business landscape.',
 '',
 '**Question 6:**',
 '',
 'Identify the core components of Data Sc

In [250]:
filtered_ques_list = [element for element in ques_list if element.endswith('?') or element.endswith('.')]

In [246]:
from langchain.chains import RetrievalQA

In [247]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_ans_gen,
                                               chain_type = "stuff",
                                               retriever = vector_store.as_retriever())

In [254]:
# Answer each question and save to a file
for question in filtered_ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    # Save answer to file
    with open("F:\\Langchain\\Interview_Ques_Creator\\research\\answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
    

Question:  Define Data Science and elaborate on its multidisciplinary nature, highlighting the key fields that contribute to its foundation.
Answer:  Data science is the study of data to extract meaningful insights for business. It's a multidisciplinary field that combines principles and practices from several key areas:

**1. Mathematics:** 
* **Statistics:**  Data science heavily relies on statistics to analyze data, identify patterns, and draw conclusions. Techniques like hypothesis testing, regression analysis, and probability theory are fundamental.
* **Linear Algebra:**  This branch of mathematics is crucial for working with large datasets, understanding relationships between variables, and implementing machine learning algorithms.
* **Calculus:**  Calculus provides the foundation for understanding optimization problems, which are often encountered in machine learning and data analysis.

**2. Computer Science:**
* **Programming:**  Data scientists need to be proficient in program